In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # Uncomment to disable GPU
from model import Model, DatasetName, load_model, remove_model
import numpy as np

__ORIG_WD__ = os.getcwd()

os.chdir(f"{__ORIG_WD__}/../data_collectors/")
from covid19_genome import Covid19Genome


os.chdir(__ORIG_WD__)



In [ ]:
covid19_genome = Covid19Genome()

In [ ]:
lineages = covid19_genome.getLocalLineages(0)
lineages.sort()
dataset = []
def get_dataset(lower, upper):
    for lineage in lineages[lower:upper]:
        dataset.append((lineage, covid19_genome.getLocalAccessionsPath(lineage)))
    return dataset

portions = {
    DatasetName.trainset.name: 0.8,
    DatasetName.validset.name: 0.1,
    DatasetName.testset.name: 0.1
}

dataset = get_dataset(0, 200)

try:
    model = Model("covid19-200")
    model.create_datasets(dataset, portions, minhash_dataset=True)
except Exception:
    model = load_model("covid19-200")


In [ ]:
# try:
model.add_ml_model("vit", "vit.1.200.01.adamw.coverage1", hps={
    "optimizer": {
        "name": "AdamW",
        "params": {
            "learning_rate": 0.001,
        },
    },
    "encoder_repeats": 1,
    "batch_size": 1024,
    "regularizer": {
        "name": "l2",
        "params": {
            "l2": 0.08
        }
    },
    "d_key": 64,
    "d_value": 64,
    "dropout": 0.3,
})
# except Exception:
    # print("Model already exists")

In [ ]:
model.list_ml_models()
# model.remove_ml_model(model.list_ml_models()[0])

In [ ]:
# model.transfer('vit.1.50.001.adamw.coverage05','vit.1.50.004.adamw.coverage1')

In [ ]:
model.change_ml_hps(
    "vit.2.50.004.adamw.coverage05",
    {
        # "optimizer": {
        #     "name": "AdamW",
        #     "params": {
        #         "learning_rate": 0.0004,
        #     },
        # },
        "regularizer": {
            "name": "l2",
            "params": {
                "l2": 0.02
            },
        },
    }
)

In [ ]:
model.set_coverage(1)
model.train("vit.1.200.01.adamw.coverage1", epochs=1000)

In [ ]:
# model.add_examples(get_dataset(200, 500), portions)